In [35]:
# GOOGLE COLAB ONLY: Install Pyomo and Ipopt
# !pip install pyomo
# !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
# !unzip -o -q ipopt-linux64

from pyomo.environ import (
    SolverFactory,
    AbstractModel,
    Param,
    Set,
    Var,
    Objective,
    Constraint,
)
from pyomo.core import NonNegativeIntegers, PositiveReals, Any, NonNegativeReals  # type: ignore

solver = SolverFactory("ipopt")

## Storage Problem/System Summary

$$
\begin{aligned}
  &\text{min} \sum_{t=0}^T w(t) \\
  &\text{s.t.} \\
  &t = \{1, 2, ..., T\} \text{ as time points} \\
  &\text{$s(t)$ as reservoir storage, $a(t)$ as surface area, $r(t)$ as rainfall, } \\
  &\text{$e(t)$ as evaporation, $q(t)$ as pumping amount, $w(t)$ as overflow at or over time $t$} \\
  &s(t + 1) = s(t) + r(t) - e(t)- q(t) - w(t), \forall t \\
  &\text{$s(t) \leq \bar S$ for maximum capacity, $a(t) \leq \bar A$ as maximum surface area, and $q(t) \leq \bar Q$ as maximum pumping rate} \\
  &r(t) = A_c p(t) \text{ where $A_c$ is a constant catchment area and $p(t)$ is precipitation} \\
  &e(t) = a(t) \cdot e_0(t) \text{ where $e_0(t)$ is the evaporation coefficient at time $t$} \\
  &s(t) = c_1 a(t) ^ {c_2} \text{(power law relationship with constants)}
\end{aligned}
$$

### Reservoir Stats

Coordinates: $36.33385606273248, -111.6578894846542$

Perimeter: $699.69$ m

Area: $19,991.22$ m\*\*2

Elevation: $1,815$ m


In [36]:
model = AbstractModel()

# Indices input parameters
model.T_intervals = Set(domain=NonNegativeIntegers)  # Time intervals
model.T_points = Set(
    domain=NonNegativeIntegers
)  # Time points, TODO: assert that this is [...T_intervals, 1]

# Constant input parameters
model.c1 = Param(within=PositiveReals, mutable=False)  # Scalar constant c1
model.c2 = Param(within=PositiveReals, mutable=False)  # Scalar constant c2
model.A_c = Param(
    within=PositiveReals, mutable=False
)  # Catchment area in square meters
model.S_max = Param(within=PositiveReals, mutable=False)  # Maximum storage capacity
model.A_max = Param(within=PositiveReals, mutable=False)  # Maximum free surface area
model.Q_max = Param(
    within=NonNegativeReals, mutable=False
)  # Upper limit for pumping rate

# Time-varying input parameters
model.P = Param(model.T_intervals)  # Precipitation
model.E_coeff = Param(model.T_intervals)  # Evaporation coefficients

# Calculated Parameters
model.R = Param(  # Rainfall
    model.T_intervals,
    default=lambda model, t: model.A_c * model.P[t],
    within=NonNegativeReals,
)

# Optimizable Variables
model.S = Var(model.T_points, domain=NonNegativeReals)  # Storage
model.A = Var(model.T_points, domain=NonNegativeReals)  # Surface area

model.E = Var(model.T_intervals, domain=NonNegativeReals)  # Evaporation
model.Q = Var(model.T_intervals, domain=NonNegativeReals)  # Pumping
model.W = Var(model.T_intervals, domain=NonNegativeReals)  # Overflow

# Objective
model.min_overflow = Objective(
    expr=lambda model: sum(model.W[t] for t in model.T_intervals)
)

# Constraints
model.upper_S = Constraint(
    model.T_points, rule=lambda model, t: model.S[t] <= model.S_max
)

model.upper_A = Constraint(
    model.T_points, rule=lambda model, t: model.A[t] <= model.A_max
)

model.upper_Q = Constraint(
    model.T_intervals, rule=lambda model, t: model.Q[t] <= model.Q_max
)

model.evaporation = Constraint(
    model.T_intervals,
    rule=lambda model, t: model.E[t] == model.E_coeff[t] ** model.A[t],
)

model.storage = Constraint(
    model.T_points, rule=lambda model, t: model.S[t] == model.c1 * model.A[t] + model.c2
)

model.dynamics = Constraint(
    model.T_intervals,
    rule=lambda model, t: model.S[t + 1]
    == model.S[t] + model.R[t] - model.E[t] - model.Q[t] - model.W[t],
)

In [37]:
# Stuff from evaporation_modeling
import pandas as pd

pan_m_df = pd.DataFrame(
    {
        "month": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        "evap": [
            0.00,
            2.60,
            5.84,
            8.27,
            10.72,
            12.86,
            13.06,
            11.38,
            8.42,
            5.13,
            2.29,
            0.00,
        ],
    }
)
pan_m_df["est"] = pan_m_df["evap"] * 25.4  # in/month -> mm/month
pan_evap = pan_m_df["est"].to_list()


def estimate_volume(area: float) -> float:
    return C1 * area**C2


SURFACE_AREA = 19991.22  # (m^2)
VOLUME = estimate_volume(SURFACE_AREA)  # (m^3)
ELEVATION = 1815  # m
LATITUDE = 36.33385606273248
CATCHMENT_AREA = 482970.15  # (m^2), from loading GEOJSON to http://geojson.io/
# C1, C2 = 0.00857, 1.4367  # Liebe, et al. (Ghana case study)
C1, C2 = 30.684, 0.978  # Lehner, et al. (Global reservoir and dam database)

In [47]:
num_intervals = 12
interval_indices = list(range(num_intervals))
points_indices = list(range(num_intervals + 1))

data = {
    None: {
        "T_intervals": {None: interval_indices},
        "T_points": {None: points_indices},
        "c1": {None: C1},
        "c2": {None: C2},
        "A_c": {None: CATCHMENT_AREA},
        "S_max": {None: VOLUME},
        "A_max": {None: SURFACE_AREA},
        "Q_max": {None: 0},
        "P": {i: evap for i, evap in enumerate(pan_evap)},
        "E_coeff": {i: 0.8 for i in interval_indices},
    }
}

instance = model.create_instance(data)
results = solver.solve(instance)
instance.pprint()

for i in instance.T_intervals:
    print(instance.S[i].value, instance.A[i].value, instance.E[i].value)

2 Set Declarations
    T_intervals : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain              : Size : Members
        None :     1 : NonNegativeIntegers :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    T_points : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain              : Size : Members
        None :     1 : NonNegativeIntegers :   13 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

9 Param Declarations
    A_c : Size=1, Index=None, Domain=PositiveReals, Default=None, Mutable=False
        Key  : Value
        None : 482970.15
    A_max : Size=1, Index=None, Domain=PositiveReals, Default=None, Mutable=False
        Key  : Value
        None : 19991.22
    E_coeff : Size=12, Index=T_intervals, Domain=Any, Default=None, Mutable=False
        Key : Value
          0 :   0.8
          1 :   0.8
          2 :   0.8
          3 :   0.8
          4 :   0.8
          5 :   0.8
          6 :   0.8
          7 :   0.8
          8 :   0.8
         